## Импорт и определение библиотек

In [1]:
# отключение параллелизма
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
# ставим разные пакеты оттуда
!pip install langchain langchain-openai langchain-community openai tiktoken langchain-huggingface -q
!pip install beautifulsoup4 pypdf sentence-transformers faiss-cpu unstructured pypandoc -q
#huggingface_hub  langchain_experimental langchainhub
#!pip install unstructured "unstructured[pdf]" -q

In [3]:
# импортируем библиотеки
from langchain.document_loaders import WebBaseLoader, PyPDFLoader, UnstructuredEPubLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
#from langchain.tools.retriever import create_retriever_tool
#from langchain.agents import initialize_agent, AgentType
from langchain.agents import Tool, initialize_agent
import tqdm as notebook_tqdm

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
# для использования ключа из LLM_bot
!wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py -q

# Готовим коктейль с помощью RAG
## Немного `Document Loader`

In [5]:
# Загрузка EPUB файла от бармена-литературоведа
loader = UnstructuredEPubLoader('../docs/Federle.epub', show_progress=True)
doc_epub = loader.load()

# смотрим, скока получилось загрузить
print('pages count:', len(doc_epub))
print('max chars on page:', max([len(page.page_content) for page in doc_epub]))

pages count: 1
max chars on page: 107261


In [ ]:
# Загружаем книгу от крутого бармена
#loader = WebBaseLoader(url)
"""loader = PyPDFLoader('../docs/Bortnik_1000.pdf', show_progress=True)
data = loader.load()

print('pages count:', len(data))
print('max chars on page:', max([len(page.page_content) for page in data]))"""

# на тот случай, если решим загружать подобную литературу пачкой pdf
"""
#!pip install unstructured "unstructured[pdf]" -q
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader("../docs", glob="**/*.pdf", show_progress=True)
data = loader.load()"""

## Исполним `Text splitters` и разобьём на чанки

In [16]:
# для gpt-3.5-turbo
splitter = RecursiveCharacterTextSplitter(
    separators=['\n\n', '\n', ' ', ''],  # Разделители для сохранения структуры
    chunk_size=1000,  # 1000 символов (~200-250 токенов)
    chunk_overlap=200  # 200 символов перекрытия
)

# для gpt-4o-mini
"""splitter = RecursiveCharacterTextSplitter(
    separators=['\n\n', '\n', ' ', ''],
    chunk_size=2000,  # 2000 символов (~400-500 токенов)
    chunk_overlap=300  # 300 символов перекрытия
)"""

# Разделяем книгу на части
#documents = splitter.split_documents(doc_epub)

split_documents = []
for page in doc_epub:
    split_documents += splitter.create_documents([page.page_content], metadatas=[page.metadata])

len(split_documents)

167

In [17]:
chunks = splitter.split_documents(doc_epub)
print('Количество чанков:', len(chunks))
print('Максимальное количество символов в чанке:', max([len(chunk.page_content) for chunk in chunks]))

Количество чанков: 167
Максимальное количество символов в чанке: 998


## Чутка `embeddings с HuggingFace`

In [8]:
# ai-sage/Giga-Embeddings-instruct
!pip install sentence-transformers -q
!pip install numpy transformers -q
#!pip install numpy -q

In [10]:
from huggingface_hub import HfApi
from langchain.embeddings import HuggingFaceEmbeddings

In [12]:
# Чтение токена из файла
def read_token(file_path):
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith("HUGGINGFACEHUB_API_TOKEN"):
                return line.split('=')[1].strip()
    raise ValueError("Token not found in the specified file.")

# Загрузка токена
token = read_token('../utils.key')

In [18]:
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",  
    #"sentence-transformers/paraphrase-multilingual-mpnet-base-v2", 
    #"cointegrated/LaBSE-en-ru",
    model_kwargs={
        "device": "cpu",
        "token": token
                  } 
)

## Наполним чашу `Vector Store`

In [19]:
# Если нужно сохранить новый документ в базу
try:
    db_embed = FAISS.from_documents(
            split_documents, 
            hf_embeddings_model
        )
except Exception as e:
    raise RuntimeError(f"Ошибка при создании базы данных FAISS: {e}")


# Сохраняем векторную базу локально
db_embed.save_local("../data/faiss_db_epub")

# Если нужно изъять из уже сохранённого документа из базы
# Загрузка векторной базы данных из локального файла
db_embed = FAISS.load_local(
        "../data/faiss_db_epub", 
        hf_embeddings_model,
        allow_dangerous_deserialization = True
    )

In [22]:
resultsdb = db_embed.similarity_search("Джейн Остин")
resultsdb

[Document(id='edec0d8b-7c49-434d-9f35-8beaca2ccafc', metadata={'source': '../docs/Federle.epub'}, page_content='Алый «Сайдкар»\n\nАЛАЯ БУКВА (1850)\n\nНАТАНИЭЛЬ ГОТОРН\n\nХотите верьте, хотите нет, но были времена, когда из-за незаконнорожденного ребенка не на реалити-шоу вели, а к позорному столбу. «Алая буква» – первая буква слова Adulteress («прелюбодейка»), которую героине следовало вышить на одежде и носить в знак того, что она запятнала честь внебрачным ребенком. В своем краеугольном романе Готорн препарирует пуританские нравы Новой Англии, где царит разлад между религиозностью, моралью, ханжеством и живым чувством. Давайте же назло поборникам традиций вместо лимонного сока добавим вишневой сладости в классический рецепт коктейля «Сайдкар» (что означает «мотоциклетная люлька»). Вы тут же убедитесь, что наш вариант бесподобен, а вам, как героине Готорна, захочется продефилировать по улицам гордой поступью и с высоко поднятой головой:\n\n30 мл вишневого сока;\n\n15 мл бренди;\n\n15

## Полирнём `Retriver` и инициализирует `llm` для вкуса

In [23]:
# Если используете ключ из курса по LLM, запустите эту ячейку
from utils import ChatOpenAI

# Считываем API ключ из файла
with open('../utils.key', 'r') as file:
    for line in file:
        if line.startswith('OPENAI_API_KEY'):
            course_api_key = line.split('=')[1].strip()
            break

# Инициализация языковой модели
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo", #"gpt-4o-mini", 
    temperature=0.2, 
    course_api_key=course_api_key
)

# объявление retriever 
retriever = db_embed.as_retriever(
        search_type="mmr",          # тип поиска похожих документов
        search_kwargs={"k": 5},     # количество возвращаемых документов (Default: 4)
        core_threshold=None,        # минимальный порог для поиска "similarity_score_threshold"
    )


retrieval_qa = RetrievalQA.from_chain_type(
        llm=llm, 
        retriever=retriever
    )

In [ ]:
"""# Предполагаем, что retriever.invoke возвращает список объектов Document
results = retriever.invoke("Джейн Остин")

# Извлекаем содержимое page_content из каждого элемента списка
formatted_contents = []

for result in results:
    # Проверяем, что результат является объектом Document и содержит page_content
    if hasattr(result, 'page_content'):
        page_content = result.page_content
        # Форматируем содержимое по абзацам
        formatted_paragraphs = "\n\n".join(paragraph.strip() for paragraph in page_content.split('\n') if paragraph.strip())
        formatted_contents.append(formatted_paragraphs)

# Объединяем все отформатированные содержимые в один текст
final_output = "\n\n".join(formatted_contents)

# Выводим окончательный результат
print(final_output)"""

# Месье системный промт

In [24]:
# Системный промт для агента
system_prompt = (
    """Ты первоклассный бармен-литературовед. 
    Со всем почтением и любовью к величайшим литераторам ты содержишь целую библиотеку коктейлей (алкогольных и безалкогольных), 
    навеянных их произведениями и хранишь её в векторной базе. По запросу ты готов приготовить коктейль из своей коллекции рецептов, который не только утолит жажду, 
    но и оживит самые яркие литературные сюжеты. Выбирай ингредиенты и расскажи, какие коктейли можно приготовить, а также чем дополнить рецепты 
    для создания новых уникальных напитков.
    Если от возлияний в голове гудит, а в животе урчит, предложи коктейли «Яйца красит Prada», «Дон Кревет» и другие «Закуски книголюба». 
    Пусти в ход наши питейные игры, после которых самое время подсчитать под столом своих истинных друзей…"""
)

# Миксуем рецепт коктейля `agent` 00х

In [25]:
# Создание инструмента
agent_tools = [
    Tool(
        name="Cocktail Recipe Finder",
        func=retrieval_qa.run,
        description="Находит рецепты коктейлей по заданным ингредиентам."
    )
]

# создание агента
agent = initialize_agent(
    tools=agent_tools, 
    llm=llm, 
    agent="zero-shot-react-description", 
    verbose=True,
    system_prompt=system_prompt         # не забыть указать системный промт для вкуса
)

/tmp/ipykernel_37267/929394129.py:11: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


# Задаём общие параметры, после чего коктейль готов

In [26]:
# Объявление функции на ввод данных от пользователя

def get_user_input():
    """
    Запрашивает у пользователя ингредиенты для поиска рецептов.
    """
    user_input = input("Введите ингредиенты, разделенные запятыми: ")
    ingredients = [ingredient.strip() for ingredient in user_input.split(",")]
    return ingredients


In [ ]:
# Формирование запроса и вывод результата

def main():
    print("Добро пожаловать в поиск коктейлей!")

    while True:
        ingredients = get_user_input()
        if not ingredients:
            print("Вы не ввели ингредиенты. Попробуйте снова.")
            continue
        
        # Формируем текстовый запрос
        user_prompt = (
                """Ты являешься помощником для выполнения заданий по созданию коктейлей. 
                Вот контекст, который нужно использовать для ответа на вопрос:
                У тебя есть коллекция коктейлей, вдохновленных великими литературными произведениями. 
                Каждый коктейль имеет свои уникальные ингредиенты и способы приготовления.
                
                Дай ответ на этот вопрос, используя только вышеуказанный контекст. 
                Для каждого коктейля напиши название и подробный способ приготовления и укажи пропорции по представленному ниже примеру. 
                Используй четкий и лаконичный формат, представленный в примере. 
                Не добавляй лишнюю информацию, которая не относится к рецепту. 
                Ответ должен содержать название коктейля, хранимое в базе, и его рецептуру.
                
                ===
                Пример:
                    коктейль "Над пропастью с ржаным" (на восемь персон):
                    - 350 мл ржаного виски;
                    - 120 мл ананасового сока;
                    - 60 мл лимонного сока;
                    - 1 л имбирного пива (истинно американский напиток, который непросто отыскать на наших просторах).
                    Виски и соки заливаем в чашу для пунша, где уже лежит глыба льда.
                ===
                Теперь просмотрите вопрос пользователя:
                Из представленных ниже ингредиентов, какие коктейли можно изготовить?"""
                f"Ингредиенты: {ingredients}.format(ingredients=", ".join(ingredients))"
                """===
                Ответ:"""
            )
        
        #user_prompt = (
        #    """Напиши, какие коктейли можно изготовить из представленных ниже ингредиентов. 
        #    Для каждого коктейля опиши подробный способ приготовления и укажи пропорции по представленному ниже примеру. 
        #    Отвечай строго на русском языке, используя четкий и лаконичный формат, представленный в примере. 
        #    Не добавляй лишнюю информацию, которая не относится к рецепту. 
        #    Ответ должен содержать название коктейля, хранимое в базе, и его рецептуру.
        #    Если в запросе указан "виски" и "сок", то ответ должен быть как в примере. 
        #    ===
        #    Пример:
        #        коктейль "Над пропастью с ржаным" (на восемь персон):
        #        - 350 мл ржаного виски;
        #        - 120 мл ананасового сока;
        #        - 60 мл лимонного сока;
        #        - 1 л имбирного пива (истинно американский напиток, который непросто отыскать на наших просторах).
        #        Виски и соки заливаем в чашу для пунша, где уже лежит глыба льда.
        #    ===
        #    Ингредиенты: {ingredients}""".format(ingredients=", ".join(ingredients))
        #)


        print("Запрашиваю информацию...")
        response = agent.run(user_prompt)
        print("\nРезультат поиска:")
        print(response)

        # Предложение продолжить или завершить
        cont = input("Хотите выполнить ещё один запрос? (да/нет): ").strip().lower()
        if cont != "да":
            print("Спасибо за использование поиска коктейлей! До свидания!")
            break

if __name__ == "__main__":
    main()


Добро пожаловать в поиск коктейлей!
Запрашиваю информацию...


> Entering new AgentExecutor chain...
Я должен найти информацию о коктейле "ОЛДОС ХАКСЛИ" и его ингредиентах. 
Action: Cocktail Recipe Finder
Action Input: "ОЛДОС ХАКСЛИ"
Observation: Не знаю.
Thought:Я не смог найти информацию о коктейле "ОЛДОС ХАКСЛИ". Возможно, его название написано неправильно или он не является популярным коктейлем. Я должен попробовать найти другие варианты или уточнить запрос. 

Action: Cocktail Recipe Finder
Action Input: "OLDOS HUXLEY"
Observation: Я не знаю.
Thought:Я не смог найти информацию о коктейле "OLDOS HUXLEY". Возможно, это редкий или нестандартный коктейль. Я должен попробовать найти другие популярные коктейли или уточнить запрос. 

Action: Cocktail Recipe Finder
Action Input: "коктейли"
Observation: В вашем запросе упоминаются несколько коктейлей. Вот некоторые из них:

1. **А**:
   - 30 мл вишневого сока
   - 15 мл бренди
   - 15 мл ликера трипл-сек
   - Сахар для украшения

   Пригото

In [32]:
"""## Если не хотите платить денежков, то запустите эту ячейку
from langchain.chains import LLMChain
from langchain_huggingface import HuggingFaceEndpoint
from langchain.embeddings import HuggingFaceEmbeddings

# Считываем ключ из файла
with open('../utils.key', 'r') as file:
    for line in file:
        if line.startswith('HUGGINGFACEHUB_API_TOKEN'):
            # Извлекаем ключ, убирая лишние пробелы и символы
            course_api_key = line.split('=')[1].strip()
            break

# Создаем объект языкового модели
llm = HuggingFaceEndpoint(
      #repo_id="IlyaGusev/saiga_llama3_8b",
      repo_id="mistralai/Mistral-7B-Instruct-v0.3",
      task="text-generation",  # Вид задачи, в нашем случае - генерация текста
      huggingfacehub_api_token = course_api_key
    )"""

In [77]:
"""messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]"""
"""# Получаем ответ от модели
response = llm(messages).content

# Выводим ответ
print("Рецепт коктейлей:\n")
print(response)"""

# Модифицируем коктейль

In [ ]:
# Используем db_embed для поиска рецептов на основе ингредиентов
def search_cocktails_in_db(ingredients_str):
    # Здесь вы можете реализовать поиск по вашей базе данных FAISS
    # Например, используя векторизацию ингредиентов и выполнение поиска
        # Предположим, что у вас есть функция для этого:
    results = db_embed.similarity_search(ingredients_str)  # Псевдокод для поиска
    return results

# Выполняем поиск в базе данных
search_results = search_cocktails_in_db(ingredients_str)

# Обрабатываем результаты поиска и формируем запрос к агенту
if search_results:
    # Извлекаем нужные данные из результатов поиска (например, текст документа)
    cocktail_names = [result.page_content for result in search_results]  # Используем page_content для получения текста
    cocktail_names_str = ", ".join(cocktail_names)
    
    # Вызываем функцию получения рецептов с найденными коктейлями
    response = get_cocktail_recipes(cocktail_names_str, iba_category)
else:
    response = "Коктейли не найдены по заданным ингредиентам."

# Выводим ответ
print("Рецепт коктейлей:\n")
print(response)

## Если используем модель с Hugging Face

In [ ]:
"""# Если выбрали модель с HuggingFace
# Объединяем системный и пользовательский промты в одну строку
full_prompt = f"{system_prompt}\n{user_prompt}"

# Получаем ответ от модели
response = llm(full_prompt)

# Выводим ответ
print(response)"""

## Меняем стиль ответа

In [ ]:
# Функция выбора стиля ответа

def choose_style():
    styles = {
        "1": "стиль космического ужаса и хтонического мрака Говарда Ф. Лавкрафта",
        "2": "гопническо-быдляцкий жаргон",
        "3": "экспериментальный стиль нарезок Уильяма Берроуза",
        "4": "Стиль барного меню"  # Стиль по умолчанию
    }
    #print("Доступные стили ответа:")
    for key, value in styles.items():
        print(f"{key}: {value}")
    chosen_style = input("Выберите номер стиля (по умолчанию 4): ") or "4"
    return styles.get(chosen_style, "style4")

style = choose_style()
prompt = PromptTemplate(input_variables=['output_text', 'style'],
                        template='''Перепиши этот текст в заданном стиле: {output_text}\nСтиль: {style}.\nРезультат:''')
style_changer_chain = prompt | llm

styled_response = style_changer_chain.invoke({'output_text': response, 'style': style}).content
print("\nОтвет в выбранном стиле:")
print(styled_response)

# Разные способы поиска цен на ингредиенты

## Поиск с помощью агента
1. Инструменты LangChain:
    - Мы можем использовать инструменты, такие как `DuckDuckGoSearchRun` (для поиска через DuckDuckGo), или создать собственные инструменты для взаимодействия с API магазинов (например, Winelab, SimpleWine и других).
2. Агент LangChain:
    - Агент будет обрабатывать запросы, отправлять их в поисковые инструменты и агрегировать ответы.
3. Поиск стоимости товара:
    - Агент использует инструменты, чтобы найти релевантную информацию о стоимости алкогольных ингредиентов.

In [ ]:
pip install -U duckduckgo-search -q

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.tools import DuckDuckGoSearchRun
from langchain.llms import OpenAI

# Подключение LLM
#llm = OpenAI(temperature=0.0, openai_api_key=course_api_key)

# Создаем поисковый инструмент DuckDuckGo
search = DuckDuckGoSearchRun()

# Добавляем инструмент в список
tools = [
    Tool(
        name="Search ingredient",
        func=search.run,
        description="Используйте этот инструмент для поиска информации об ингредиентах и ценах на самые популярные из них."
    )
]

# Инициализация агента
price_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",  # Используем Zero-shot агент
    verbose=True
)

# Формируем промт для поиска цен на товар
ingredient = "бурбон"
city = "Санкт-Петербург"
prompt = (
    f"Найдите цену на продукт '{ingredient}' в интернет-магазинах {city}. "
    """Если вариантов несколько, укажите 3 лучших результата. 
    Если в ответе название пишется на языке производителя, то оставь как есть. Название валюты и объём должны быть указаны на русском языке. 
    Укажи ответ в формате: название экземпляра, стоимость в среднем ценовом диапазоне, объём, ссылка на магазин, если имеется; если нет, то ничего не писать. 
    Если вариантов ответа несколько, то отобрази их в виде вертикального списка, где каждый новый вариант отображается с новой строки. """
)

# Выполнение агента
result = price_agent.run(prompt)

print(result)


## Поиск с помощью `RAG`

Общая стратегия:

1. Использование `WebLoader` для сбора данных:
   - скачиваем страницы для каждой категории алкоголя с сайтов, таких как `Winelab`.
   - сохраняем их текстовое содержимое (например, названия и цены продуктов) в векторное хранилище.
1. Создание векторного хранилища:
    - разбиваем загруженные данные на чанки.
    - индексируем их с помощью векторизатора (например, FAISS или Chroma).
1. Поиск ингредиентов:
    - при запросе пользователя (например, "водка"), извлекаем соответствующую информацию из хранилища.
1. Использование `LangChain`:
    - для загрузки данных используем WebBaseLoader из LangChain.
    - для индексации данных — инструменты LangChain, такие как Chroma или FAISS.
    - для обработки запросов пользователя — цепочки, основанные на LLM (например, `OpenAI API`).

In [ ]:
pip install bs4 langchain-openai faiss-cpu -q

In [ ]:
import os
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

# Установите ваш ключ API
# Считываем ключ из файла
with open('../utils.key', 'r') as file:
    for line in file:
        if line.startswith('OPENAI_API_KEY'):
            # Извлекаем ключ, убирая лишние пробелы и символы
            course_api_key = line.split('=')[1].strip()
            break

# 1. Сбор данных с категорий сайта
def fetch_and_index_alcohol_data():
    categories = {
        "absinthe": "https://www.winelab.ru/catalog/krepkiy-alkogol-absent",
        "vodka": "https://www.winelab.ru/catalog/krepkiy-alkogol-vodka",
        "whiskey": "https://www.winelab.ru/catalog/krepkiy-alkogol-viski",
    }

    documents = []
    for category, url in categories.items():
        try:
            loader = WebBaseLoader(url)
            category_docs = loader.load()
            documents.extend(category_docs)
            print(f"Загружено {len(category_docs)} документов из {url}.")
        except Exception as e:
            print(f"Ошибка при загрузке данных из {url}: {e}")

    print(f"Всего загружено документов: {len(documents)}")
    return documents

# 2. Разбивка текста на чанки
def split_documents(documents):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = splitter.split_documents(documents)
    print(f"Всего создано чанков: {len(chunks)}")
    return chunks


# 3. Индексация данных с использованием FAISS
def create_faiss_index(chunks):
    if not chunks:
        raise ValueError("Нет данных для индексации. Проверьте загрузку и разбивку документов.")
    
    embeddings = OpenAIEmbeddings(openai_api_key=course_api_key)
    vectorstore = FAISS.from_documents(chunks, embeddings)
    return vectorstore


# 4. Создание цепочки RAG для поиска
def create_retrieval_chain(vectorstore):
    retriever = vectorstore.as_retriever()
    llm = OpenAI(temperature=0.0, openai_api_key=course_api_key)
    chain = RetrievalQA(llm=llm, retriever=retriever)
    return chain

# Основная функция
def main():
    print("Сбор данных...")
    documents = fetch_and_index_alcohol_data()
    print("Разделение на чанки...")
    chunks = split_documents(documents)
    print("Создание векторного индекса...")
    vectorstore = create_faiss_index(chunks)
    print("Создание цепочки RAG...")
    rag_chain = create_retrieval_chain(vectorstore)

    # Поиск цены ингредиента
    ingredient_query = "водка"
    print(f"Поиск для: {ingredient_query}")
    result = rag_chain.run(ingredient_query)
    print(f"Результат: {result}")

if __name__ == "__main__":
    main()

## Поиск `в лоб`

In [ ]:
import requests
import re

def search_price_with_specific_cookies(ingredient):
    """
    Функция для поиска цен на алкогольные ингредиенты с индивидуальными куки для каждого магазина.
    """
    stores = [
        {
            "name": "Winelab",
            "url": lambda ing: f"https://www.winelab.ru/search/facets?text={ing}",
            "cookies": {"age-confirmed": "1"},
            "title_pattern": r'<a class="product-card__link" .*? title="(.*?)"',
            "price_pattern": r'<div class="price__main-value">(.*?)</div>',
            "link_pattern": r'<a class="product-card__link" href="(.*?)"',
            "link_prefix": "https://www.winelab.ru"
        },
        {
            "name": "SimpleWine",
            "url": lambda ing: f"https://simplewine.ru/catalog/?q={ing}",
            "cookies": {"age-confirm": "1"},
            "title_pattern": r'<a class="product-card__title-link" .*?>(.*?)</a>',
            "price_pattern": r'<div class="product-card__price-current">(.*?)</div>',
            "link_pattern": r'<a class="product-card__title-link" href="(.*?)"',
            "link_prefix": "https://simplewine.ru"
        }
    ]

    results = []

    for store in stores:
        try:
            headers = {
                "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:133.0) Gecko/20100101 Firefox/133.0"
            }
            response = requests.get(store["url"](ingredient), headers=headers, cookies=store["cookies"], timeout=5)
            if response.status_code == 200:
                # Извлечение данных о товарах
                titles = re.findall(store["title_pattern"], response.text)
                prices = re.findall(store["price_pattern"], response.text)
                links = re.findall(store["link_pattern"], response.text)

                for title, price, link in zip(titles[:3], prices[:3], links[:3]):
                    full_link = f"{store['link_prefix']}{link}"
                    results.append(f"- {title.strip()} {price.strip()} рублей {full_link} ({store['name']})")

                if results:
                    break  # Если найдены результаты, завершаем поиск
            else:
                results.append(f"Ошибка доступа к {store['name']} для {ingredient}: статус {response.status_code}")
        except Exception as e:
            results.append(f"Ошибка при выполнении поиска в {store['name']} для {ingredient}: {str(e)}")

    if not results:
        results.append(f"Данные о наличии в продаже {ingredient} не найдены.")

    return "\n".join(results)

# Пример использования
ingredient_name = "джин"
print(search_price_with_specific_cookies(ingredient_name))

In [ ]:
# Разбор коктейля и ингредиентов для поиска цен
pattern = r"\*\*(.*?)\*\*:.*?((?:\n\s*-\s*.*?\n)+)"
match = re.search(pattern, response, re.DOTALL)
if match:
    cocktail_name = match.group(1)
    ingredients_section = match.group(2)
    print(f"\nКоктейль: {cocktail_name}")
    print(f"Ингредиенты:")
    print(ingredients_section)
    
    # Поиск цен для каждого ингредиента
    ingredient_lines = ingredients_section.strip().split("\n")
    for line in ingredient_lines:
        ingredient_match = re.search(r"-\s*\d+\s*мл\s*([^\n]+)", line)
        if ingredient_match:
            ingredient_name = ingredient_match.group(1)
            print(f"\nИщем цену для: {ingredient_name}")
            prices = search_price(ingredient_name)
            print(prices)
        else:
            print(f"\nНе удалось извлечь название ингредиента из строки: {line}")
else:
    print("Не удалось найти информацию о коктейле и ингредиентах в ответе модели.")

## Поиск с помощью Selenium

In [ ]:
pip install selenium -q

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def search_price_with_selenium(ingredient):
    """
    Поиск цен на ингредиенты с использованием Selenium.
    """
    # Укажите путь к вашему WebDriver
    service = Service('/home/elijah/Documents/chromedriver-linux64/chromedriver')  # Укажите корректный путь к ChromeDriver
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Запуск в безголовом режиме
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")

    driver = webdriver.Chrome(service=service, options=chrome_options)

    results = []
    try:
        # Магазин Winelаб
        driver.get("https://www.winelab.ru")

        # Подтверждение возраста
        age_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "age-confirm"))
        )
        age_button.click()

        # Поиск ингредиента
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "text"))  # Обратите внимание на имя поля
        )

        # Добавление небольшой задержки для обеспечения доступности элемента
        time.sleep(1)

        # Используем JavaScript для установки значения в поле ввода
        driver.execute_script("arguments[0].value = arguments[1];", search_box, ingredient)
        
        # Теперь вызываем событие input для имитации ввода текста
        search_box.send_keys(Keys.RETURN)

        # Найдите кнопку отправки и нажмите ее (если это необходимо)
        # search_button = driver.find_element(By.CLASS_NAME, "js_search_button")
        # search_button.click()

        # Сбор результатов
        products = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "product-card"))
        )
        
        for product in products[:3]:  # Берём максимум 3 результата
            title = product.find_element(By.CLASS_NAME, "product-card__title").text
            price = product.find_element(By.CLASS_NAME, "price__main-value").text
            
            results.append(f"- {title} {price.strip()} рублей")

    except Exception as e:
        results.append(f"Ошибка при обработке ингредиента {ingredient}: {str(e)}")
    finally:
        driver.quit()

    return results if results else f"Данные о наличии в продаже {ingredient} не найдены."

# Пример использования
ingredient_name = "водка"
print(search_price_with_selenium(ingredient_name))